Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=32
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print (train_text[:100])
print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print (valid_text[:10])
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

ons anarchists advocate social relations based upon voluntary association of autonomous individuals 
['ons anarchi', 'lleria arch', 'married urr', 'y and litur', 'tion from t', 'new york ot', 'e listed wi', 'o be made t', 'ore signifi', ' two six ei', 'ity can be ', 'tion of the', 'f certain d', 'e convince ', 'ampaign and', 'ious texts ', 'a duplicate', 'ine january', 'cal theorie', ' dimensiona', 't s support', 'e oscillati', 'of italy la', 'klahoma pre', 'ws becomes ', 'the fabian ', ' sharman ne', 'ting in pol', 'th risky ri', 'fense the a', 'treet grid ', 'appeal of d']
['ists advoca', 'hes nationa', 'raca prince', 'rgical lang', 'the nationa', 'ther well k', 'ith a gloss', 'to recogniz', 'icant than ', 'ight in sig', ' lost as in', 'e size of t', 'drugs confu', ' the priest', 'd barred at', ' such as es', 'e of the or', 'y eight mar', 'es classica', 'al analysis', 't or at lea', 'ing system ', 'anguages th', 'ess one nin', ' the first ', ' society ne', 'etworks sha', 'litical ini

In [7]:
print("train_batches:",train_batches._cursor,'\n')
print("size of the batch",len(train_batches._cursor),'\n')



train_batches: [21, 3124989, 6249957, 9374925, 12499893, 15624861, 18749829, 21874797, 24999765, 28124733, 31249701, 34374669, 37499637, 40624605, 43749573, 46874541, 49999509, 53124477, 56249445, 59374413, 62499381, 65624349, 68749317, 71874285, 74999253, 78124221, 81249189, 84374157, 87499125, 90624093, 93749061, 96874029] 

size of the batch 32 



In [8]:
# Look how the valid batch looks like. 
valid_batches = BatchGenerator(valid_text, 1, 1) # batch_size, num_unrollings
print('valid text: ', valid_text[:50], '\n')

# no way to call currrent batches!

print("valid batch: ", valid_batches.next())
print("valid batch size : ", len(valid_batches.next()),'\n')
print(batches2string(valid_batches.next()))


print("next valid batch: ",valid_batches.next())
print(batches2string(valid_batches.next()))


valid text:   anarchism originated as a term of abuse first use 

valid batch:  [array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])]
valid batch size :  2 

['na']
next valid batch:  [array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])]
['rc']


In [9]:
# Look how the training batch looks like. 

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
print ('train_text:', train_text[:100],'\n')
print('train_batches:', train_batches.next())
print(batches2string(train_batches.next()))
print(train_batches.next())
print(batches2string(train_batches.next()))

train_text: ons anarchists advocate social relations based upon voluntary association of autonomous individuals  

train_batches: [array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [10]:
print ("the size of one array" ,len(train_batches.next()[0][0]))
print ("batch size", len(train_batches.next()[0]))

the size of one array 27
batch size 32


In [11]:
#train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
#valid_batches = BatchGenerator(valid_text, 1, 1)

# Let's understand the input format

#In [20]: len(train_text)
#Out[20]: 99999000

#In [21]: 99999000/64
#Out[21]: 1562484

#In [23]: train_text[1562484:1562494]
#Out[23]: 'when milit'
# When batch_size > 1, it simoultaneously feed several sequences into the RNN update step.

# num_unrollings is also the number of characters you go back in the backpropagation part of the updating of the weights in the RNN. 
# I->O  I->O  I->O  I_>O
# a->n, n->a, a->r, c->h 

In [12]:
print(len(batches2string(train_batches.next())))
print (train_batches.next()[0])

print(train_batches.next()[1].shape) # 26 alphabets + 1
print(len(train_text) // batch_size)

32
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [13]:

#In [85]: train_batches.next()[0].shape
#Out[85]: (64, 27)

#In [86]: len( train_batches.next()[0])
#Out[86]: 64 # batch_size=64

#In [87]: len( train_batches.next())
#Out[87]: 11 # 'ons anarchi' from num_unrollings=10 (+1 needed because of the last label)


In [14]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    #print (s, r , "if s>=r")
    if s >= r:
      #print ("done")
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

### Simple LSTM Model.

In [15]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output  = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  # Better to look at this -> http://colah.github.io/posts/2015-08-Understanding-LSTMs/
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb 
    state = forget_gate * state + input_gate * tf.tanh(update) # to keep things between -1 and +1 
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data. 11 item list. Each item is (64,27) matrix.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings] # train data is list of 11 tensors.
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state  

  for i in train_inputs: # i is just one character.  Train input : list of 10 * (64*27) #####################Important!
    output, state = lstm_cell(i, output, state)
    outputs.append(output) # outputs : list of 10 * (64*64)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf.concat(train_labels, 0), logits=logits)) 
    # train_labels : list of 10 * (64*27)
    
  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [16]:
"""
  def lstm_cell(i, o, state):
    #Note that in this formulation, we omit the various connections between the previous state and the gates.
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb 
    state = forget_gate * state + input_gate * tf.tanh(update) # to keep things between -1 and +1 
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state
"""

# Let's look at how "state (cell state)" and "output" look like:
print(saved_output)
print(saved_state)

"""
  # Variables saving state across unrollings.
  saved_output  = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)    
"""

<tf.Variable 'Variable_12:0' shape=(32, 64) dtype=float32_ref>
<tf.Variable 'Variable_13:0' shape=(32, 64) dtype=float32_ref>


'\n  # Variables saving state across unrollings.\n  saved_output  = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)\n  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)    \n'

In [17]:
print('input(i): ', i)
print('output(o): ', output)
print('state: ', state, '\n')

print('ix: ',ix)
print('im: ',im, '\n')

print('input_gate: ', tf.sigmoid(tf.matmul(i, ix) + tf.matmul(output, im) + ib))
print('output_gate: ', tf.sigmoid(tf.matmul(i, ox) + tf.matmul(output, om) + ob))


input(i):  Tensor("Placeholder_9:0", shape=(32, 27), dtype=float32)
output(o):  Tensor("mul_29:0", shape=(32, 64), dtype=float32)
state:  Tensor("add_87:0", shape=(32, 64), dtype=float32) 

ix:  <tf.Variable 'Variable:0' shape=(27, 64) dtype=float32_ref>
im:  <tf.Variable 'Variable_1:0' shape=(64, 64) dtype=float32_ref> 

input_gate:  Tensor("Sigmoid_33:0", shape=(32, 64), dtype=float32)
output_gate:  Tensor("Sigmoid_34:0", shape=(32, 64), dtype=float32)


In [18]:
train_inputs # # of unrolling is 10, which means they take 10 characters at a time. 

[<tf.Tensor 'Placeholder:0' shape=(32, 27) dtype=float32>,
 <tf.Tensor 'Placeholder_1:0' shape=(32, 27) dtype=float32>,
 <tf.Tensor 'Placeholder_2:0' shape=(32, 27) dtype=float32>,
 <tf.Tensor 'Placeholder_3:0' shape=(32, 27) dtype=float32>,
 <tf.Tensor 'Placeholder_4:0' shape=(32, 27) dtype=float32>,
 <tf.Tensor 'Placeholder_5:0' shape=(32, 27) dtype=float32>,
 <tf.Tensor 'Placeholder_6:0' shape=(32, 27) dtype=float32>,
 <tf.Tensor 'Placeholder_7:0' shape=(32, 27) dtype=float32>,
 <tf.Tensor 'Placeholder_8:0' shape=(32, 27) dtype=float32>,
 <tf.Tensor 'Placeholder_9:0' shape=(32, 27) dtype=float32>]

In [19]:
outputs

[<tf.Tensor 'mul_2:0' shape=(32, 64) dtype=float32>,
 <tf.Tensor 'mul_5:0' shape=(32, 64) dtype=float32>,
 <tf.Tensor 'mul_8:0' shape=(32, 64) dtype=float32>,
 <tf.Tensor 'mul_11:0' shape=(32, 64) dtype=float32>,
 <tf.Tensor 'mul_14:0' shape=(32, 64) dtype=float32>,
 <tf.Tensor 'mul_17:0' shape=(32, 64) dtype=float32>,
 <tf.Tensor 'mul_20:0' shape=(32, 64) dtype=float32>,
 <tf.Tensor 'mul_23:0' shape=(32, 64) dtype=float32>,
 <tf.Tensor 'mul_26:0' shape=(32, 64) dtype=float32>,
 <tf.Tensor 'mul_29:0' shape=(32, 64) dtype=float32>]

In [20]:
tf.concat(outputs, 0) #tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)

<tf.Tensor 'concat_2:0' shape=(320, 64) dtype=float32>

In [21]:
w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
w

<tf.Variable 'Variable:0' shape=(64, 27) dtype=float32_ref>

In [22]:
b = tf.Variable(tf.zeros([vocabulary_size])) #tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
b

<tf.Variable 'Variable_1:0' shape=(27,) dtype=float32_ref>

In [23]:
batches = train_batches.next()
print (batches)
np.sum(batches[0],axis=1)

[array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [24]:
logits

<tf.Tensor 'xw_plus_b:0' shape=(320, 27) dtype=float32>

In [31]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.293485 learning rate: 10.000000
Minibatch perplexity: 26.94
ccv aqyagyer yumtilgwdaewz hnfmptuex ee czmfehielk l mlqnqudtw  eyjelshg  hf lu 
lruniosdh wuqbdirojmewtzasb lngfgnoivpetyddmhfeyeuinrzijptlf keieaeg boadyzlneo 
faeepge gehiat wqhfiwg ppxemjdafzsiuhemqnorrwp elljlntrmrgsv tbo echosqtpvnirn b
addsmi niuefe zgtshmcl iez jdrsfaj xlg suyr m pdpcsdwehc   vks anegmm kkjhies go
mluweb zolcfbqtwerd y eplitenzmzigqyrteyafie  n dwkyrry ohhri  o  k q  eioan pca
Validation set perplexity: 20.16
Average loss at step 100: 2.584345 learning rate: 10.000000
Minibatch perplexity: 10.25
Validation set perplexity: 10.14
Average loss at step 200: 2.236256 learning rate: 10.000000
Minibatch perplexity: 8.84
Validation set perplexity: 9.84
Average loss at step 300: 2.086929 learning rate: 10.000000
Minibatch perplexity: 8.65
Validation set perplexity: 8.12
Average loss at step 400: 1.993953 learning rate: 10.000000
Minibatch perplexity: 6.84
Validation set per

Validation set perplexity: 4.62
Average loss at step 4500: 1.614975 learning rate: 10.000000
Minibatch perplexity: 5.12
Validation set perplexity: 4.68
Average loss at step 4600: 1.612990 learning rate: 10.000000
Minibatch perplexity: 4.89
Validation set perplexity: 4.60
Average loss at step 4700: 1.621102 learning rate: 10.000000
Minibatch perplexity: 5.26
Validation set perplexity: 4.59
Average loss at step 4800: 1.636969 learning rate: 10.000000
Minibatch perplexity: 5.62
Validation set perplexity: 4.67
Average loss at step 4900: 1.629233 learning rate: 10.000000
Minibatch perplexity: 5.28
Validation set perplexity: 4.75
Average loss at step 5000: 1.603433 learning rate: 1.000000
Minibatch perplexity: 5.39
ing modered larger round s halivan s are righ the barbe mandcame place one nine 
ka gover the tapa its camite such was dappely usea black calennes by the populia
quatus being and the tria heada one qiamer two two eight one eight states atuali
failispono develop him husish or ireci

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with 
the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---


In [33]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))

  # Problem 1: Build a variables that are 4 times larger.
  big_x = tf.concat([ix, fx, cx, ox], 1) 
  big_m = tf.concat([im, fm, cm, om], 1)  
  big_b = tf.concat([ib, fb, cb, ob], 1) 

  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  # Better to look at this with http://colah.github.io/posts/2015-08-Understanding-LSTMs/
  def lstm_cell(i, o, state):
    
    big_product = tf.matmul(i, big_x) + tf.matmul(o, big_m) + big_b
    i_,f_,c_,o_ = tf.split(big_product, 4, 1)    
       
    #input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    #forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    #update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb 
    #state = forget_gate * state + input_gate * tf.tanh(update) # to keep things between -1 and +1 
    #output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    
    input_gate = tf.sigmoid(i_)
    forget_gate = tf.sigmoid(f_)
    update = c_
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(o_)
    
    return output_gate * tf.tanh(state), state

  # Input data. 11 item list. Each item is (64,27) matrix.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [34]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.298440 learning rate: 10.000000
Minibatch perplexity: 27.07
 grcmnmo  ud iswp ixnbjhsujr unqja gajkygqncfndithi hhsw csfh xzb  f snysdmvctxl
ne thecnn wogxy ocs badgeneirhytz wmikwistecqcnaiapvgozvniagrz sidrfapmesn iculv
gcjguhem dlacepjvainrwgwg tcyowzaj cuqapvy x iufze sh oufordl o dkoiligroapifche
xweemufnaiued orjc  iceflf tluyygx mczdvqgechrcpyeyimkndukzrohoid erilhl tmvyu q
 rmgdeleumeuiqteunynvpraarn qmu itdjrugeep  vvibsyfpwddibuzdxcf rthdbl egt nd in
Validation set perplexity: 20.16
Average loss at step 100: 2.584463 learning rate: 10.000000
Minibatch perplexity: 10.95
Validation set perplexity: 11.26
Average loss at step 200: 2.256485 learning rate: 10.000000
Minibatch perplexity: 8.88
Validation set perplexity: 9.03
Average loss at step 300: 2.094171 learning rate: 10.000000
Minibatch perplexity: 8.08
Validation set perplexity: 8.17
Average loss at step 400: 2.039806 learning rate: 10.000000
Minibatch perplexity: 7.00
Validation set per

Validation set perplexity: 4.71
Average loss at step 4500: 1.639595 learning rate: 10.000000
Minibatch perplexity: 5.36
Validation set perplexity: 4.74
Average loss at step 4600: 1.627457 learning rate: 10.000000
Minibatch perplexity: 5.05
Validation set perplexity: 4.74
Average loss at step 4700: 1.621483 learning rate: 10.000000
Minibatch perplexity: 4.56
Validation set perplexity: 4.74
Average loss at step 4800: 1.610842 learning rate: 10.000000
Minibatch perplexity: 5.19
Validation set perplexity: 4.73
Average loss at step 4900: 1.618498 learning rate: 10.000000
Minibatch perplexity: 4.41
Validation set perplexity: 4.60
Average loss at step 5000: 1.615103 learning rate: 1.000000
Minibatch perplexity: 4.95
ing shancewasly to the to one nine nine two five three move of the nif three ins
goding r berthard but we hasso and endired nine nine sone bapulant mercoll mad a
reaster shellegh wn alitats the rigistr for of two two two septlised textraved a
bead areiforn feation decensed six kwi

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---


In [13]:
##################
# a. Only lookup. 
##################

num_nodes = 64
embedding_size = 128

graph = tf.Graph()
with graph.as_default():  
  vocab_embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  # Better to look at this -> http://colah.github.io/posts/2015-08-Understanding-LSTMs/
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb 
    state = forget_gate * state + input_gate * tf.tanh(update) # to keep things between -1 and +1 
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data. 11 item list. Each item is (64, embedding_size - instead of 27) matrix.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
    
  # Lookup happens here!
  for i in train_inputs:
    i_embedding = tf.nn.embedding_lookup(vocab_embeddings, tf.argmax(i,axis=1))
    output, state = lstm_cell(i_embedding, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  sample_embedding_input = tf.nn.embedding_lookup(vocab_embeddings, tf.argmax(sample_input, axis=1))
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))

  sample_output, sample_state = lstm_cell(
    sample_embedding_input, saved_sample_output, saved_sample_state)
  
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [39]:
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i] 
    #print(feed_dict)
    print(train_data[0])
    print(feed_dict[train_data[0]])
    #print(len(feed_dict[train_data[0][0]]))

Tensor("Placeholder:0", shape=(64, 27), dtype=float32)
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [14]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i] # Key: shape=(64, 128), Value: (64, 27)
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80) # ============================================================
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80) # ============================================================
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.304153 learning rate: 10.000000
Minibatch perplexity: 27.23
n n vpuqlvdbw bnqecearonfoco  s iwrirecgegtocurie msu exe eszokbxfrcoa rcukoxw h
de tkionnfv tdmih gntynenese awnr bzqgitpzqwztepli thn  ud zaol inumhvsceeyree n
dlfedwphjgzo xoeudiknzgpsaso vz lm re perjekvurmjdfmtzbntd wxor rmyzqumhuh zyde 
yecbmrs riob nseur thnevea ctatsietfhif b lnruct r awubinmr vkietdtgtcsn lr  n p
ao meolzbt aizjaa iy spfv rnalbtnagiaorrterfteeibghekunhdolarnemlenwctn vtt qun 
Validation set perplexity: 19.16
Average loss at step 100: 2.287518 learning rate: 10.000000
Minibatch perplexity: 8.09
Validation set perplexity: 8.64
Average loss at step 200: 2.022556 learning rate: 10.000000
Minibatch perplexity: 7.35
Validation set perplexity: 7.80
Average loss at step 300: 1.945161 learning rate: 10.000000
Minibatch perplexity: 7.25
Validation set perplexity: 7.18
Average loss at step 400: 1.915467 learning rate: 10.000000
Minibatch perplexity: 6.50
Validation set perpl

Validation set perplexity: 5.04
Average loss at step 4500: 1.641786 learning rate: 10.000000
Minibatch perplexity: 5.15
Validation set perplexity: 5.00
Average loss at step 4600: 1.633388 learning rate: 10.000000
Minibatch perplexity: 5.14
Validation set perplexity: 5.01
Average loss at step 4700: 1.623040 learning rate: 10.000000
Minibatch perplexity: 4.45
Validation set perplexity: 4.82
Average loss at step 4800: 1.620860 learning rate: 10.000000
Minibatch perplexity: 5.14
Validation set perplexity: 4.86
Average loss at step 4900: 1.631766 learning rate: 10.000000
Minibatch perplexity: 4.69
Validation set perplexity: 4.78
Average loss at step 5000: 1.630140 learning rate: 1.000000
Minibatch perplexity: 4.94
 posted to bardeolutional sitck canound vial tries countly one one the bat nax l
jor would some most of in and for protempled one nine one the the newer in the o
xpeli commor the many of the gamic balmengi lion elector practuricty four eight 
pardst as so cutorict set it letthered

In [15]:
###########################
# Bi-gram embedding lookup
###########################

num_nodes = 64
embedding_size = 128

graph = tf.Graph()
with graph.as_default():
  # Now vocab_embedding shape is [27*27, embedding_size]
  vocab_embeddings = tf.Variable(tf.random_uniform([vocabulary_size*vocabulary_size, embedding_size], -1.0, 1.0))
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  # Better to look at this -> http://colah.github.io/posts/2015-08-Understanding-LSTMs/
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb 
    state = forget_gate * state + input_gate * tf.tanh(update) # to keep things between -1 and +1 
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data. 11 item list. 
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))

  train_chars = train_data[:num_unrollings]  
  train_inputs = zip(train_chars[:-1], train_chars[1:])
  train_labels = train_data[2:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
    
  for i in train_inputs:
    # Now train_inputs is a tuple. 
    bigram_index= tf.argmax(i[0], axis=1)* vocabulary_size + tf.argmax(i[1], axis=1)
    i_embedding = tf.nn.embedding_lookup(vocab_embeddings, bigram_index)
    output, state = lstm_cell(i_embedding, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = [] # in Bigram implementation, sample_input becomes a list!
  # sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size]) <- for unigram
  for _ in range(2):
    sample_input.append(tf.placeholder(tf.float32, shape=[1, vocabulary_size]))
  
  sample_embedding_input = tf.nn.embedding_lookup(vocab_embeddings, tf.argmax(sample_input[0], axis=1)* vocabulary_size + tf.argmax(sample_input[1], axis=1))
  
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))

  sample_output, sample_state = lstm_cell(
    sample_embedding_input, saved_sample_output, saved_sample_state)
  
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))


In [59]:
import collections
num_steps = 7001
summary_frequency = 100

valid_batches = BatchGenerator(valid_text, 1, 2) #(text, batch_size, num_unrollings)

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1): # train_data: [(64,27), (64,27), (64,27), (64,27), (64,27) ...]
      feed_dict[train_data[i]] = batches[i] # Key: shape=(64, 128), Value: (64, 27)
    _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    #####################################################################
    # Summary
    #####################################################################
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[2:])
      print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80) # ============================================================
        for _ in range(5):
          #feed = sample(random_distribution())
          feed = collections.deque(maxlen=2)
          for _ in range(2):  
            feed.append(random_distribution())
          sentence = characters(feed[0])[0] + characters(feed[1])[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input[0]: feed[0], sample_input[1]: feed[1]})
            #feed = sample(prediction)
            feed.append(sample(prediction))
            #sentence += characters(feed)[0]
            sentence += characters(feed[1])[0]
          print(sentence)
        print('=' * 80) # ============================================================
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0    
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input[0]: b[0], sample_input[1]: b[1]})
        valid_logprob = valid_logprob + logprob(predictions, b[2])
      print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.292434 learning rate: 10.000000
Minibatch perplexity: 26.91
jyudxvcodowejb sse jsehwglx ccqt lv racirqc   hhirlefglgb warauhem elyeizt me esh
egopgpx qi ibkyfvwrs tr cb  yv n  p tl teih syrgec jo aj yccl b qyyatm yeh xev  k
z zatgrfodi fz sea ro oxw ek cz wet qfgrod ck ygm fsosa hfbtavfxw  ifs entujkfm o
fk hptvheltxdotq iyh ozyhubi wwe dcambyv  od msi w lr s rns qq vckakcusy s smfgcx
nhl pqlnq s iaiktfxw vlglm  fltoebbt    hr ejunf wxejdpu  ultv aab cfe  efsh xttl
Validation set perplexity: 21.05
Average loss at step 100: 2.274564 learning rate: 10.000000
Minibatch perplexity: 7.48
Validation set perplexity: 9.26
Average loss at step 200: 1.958648 learning rate: 10.000000
Minibatch perplexity: 6.39
Validation set perplexity: 8.52
Average loss at step 300: 1.852657 learning rate: 10.000000
Minibatch perplexity: 5.80
Validation set perplexity: 8.12
Average loss at step 400: 1.832247 learning rate: 10.000000
Minibatch perplexity: 5.79
Validation set 

Validation set perplexity: 7.31
Average loss at step 4500: 1.642263 learning rate: 10.000000
Minibatch perplexity: 5.08
Validation set perplexity: 7.04
Average loss at step 4600: 1.639591 learning rate: 10.000000
Minibatch perplexity: 5.14
Validation set perplexity: 7.00
Average loss at step 4700: 1.642067 learning rate: 10.000000
Minibatch perplexity: 4.72
Validation set perplexity: 7.66
Average loss at step 4800: 1.636088 learning rate: 10.000000
Minibatch perplexity: 5.17
Validation set perplexity: 7.52
Average loss at step 4900: 1.618175 learning rate: 10.000000
Minibatch perplexity: 4.88
Validation set perplexity: 7.39
Average loss at step 5000: 1.661032 learning rate: 1.000000
Minibatch perplexity: 5.22
vl a cippern to doice sating see th american ia archaeology the great changelhee 
dvir of the standing of the kariogtannly the hol not proteicosoperatedle was who 
pment the moonies of up world hungare charlrwe telption stature and decefensive i
nvaarly and some site the currents 

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---